In [1]:
import pandas as pd
import os
import socks
import socket
import urllib2 as urllib
import time
from bs4 import BeautifulSoup
from multiprocessing import Pool
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

TIMEOUT = 1.0 # 1 second
N_PROC = 16

targets = map(lambda x : x.lower(),
              ["Trump", 
               "Cruz", 
               "Rubio", 
               "Carson",
               "Paul",
               "Bush",
               "Huckabee",
               "Kasich",
               "Fiorina",
               "Christie",
               "Santorum",
               "O'Malley",
               "Clinton", 
               "Sanders"])

In [19]:
def create_connection(address, timeout=None, source_address=None):
    sock = socks.socksocket()
    sock.connect(address)
    return sock

socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9050)

# patch the socket module
socket.socket = socks.socksocket
socket.create_connection = create_connection

def extract_freqs(targets, url):
    start_time = time.clock()
    try:
        html = urllib.urlopen(url, timeout = TIMEOUT).read()
        soup = BeautifulSoup(html)

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()
        
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines

        word_list = [x.lower()
                     for chunk in chunks if chunk 
                     for x in chunk.split(" ")]

        counts = dict(zip(targets, [0 for i in xrange(len(targets))]))

        for word in word_list:
            if word in counts:
                counts[word] += 1
        return counts, len(word_list), time.clock() - start_time
    
    except:
        return None, None, time.clock() - start_time

def extract_freqs_(args):
    return extract_freqs(args[0], args[1])

In [ ]:
file_name = '20130401.export.CSV'

p = Pool(N_PROC)

for file_name in os.listdir('./'):
    if '.CSV' not in file_name:
        continue
    event_table = pd.read_csv(file_name, sep = '\t', header = None)
    print 'Processing ' + str(file_name)
    count_table['counts'] = p.map(extract_freqs_,
                                  zip([targets] * len(event_table), 
                                      event_table[57].values))
    print 'Done processing ' + str(file_name)
    count_table.to_csv('counts_' + file_name)